In [1]:
!pip install datasets lxml TinyImageNet --quiet

In [2]:
import torch
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from scipy.stats import wasserstein_distance
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AffinityPropagation
import matplotlib.pyplot as plt
import random
import seaborn as sns
import copy
import csv
import gc
import logging
import os
import os.path
import random
import tarfile
import warnings
from collections import OrderedDict
from datetime import datetime
from functools import partial
from math import sqrt
from typing import Callable, List, Optional

import matplotlib.pyplot as plt
import numpy as np
import PIL
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from scipy.spatial.distance import cosine, euclidean, jensenshannon
from scipy.stats import wasserstein_distance
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from tinyimagenet import TinyImageNet
from torch import nn
from torch.autograd import Variable
from torch.utils.model_zoo import tqdm
from torchvision.datasets import (CIFAR10, CIFAR100, MNIST, STL10, SVHN,
                                  DatasetFolder, FashionMNIST, ImageFolder)
from torchvision.datasets.utils import (check_integrity,
                                        download_file_from_google_drive)
from torchvision.datasets.vision import VisionDataset
from torchvision.transforms import Normalize

In [3]:
seed = 1
random.seed(seed)
np.random.seed(seed)

In [4]:
def get_top_values(arr, p):
    # Ensure p is a valid percentage
    if not (0 <= p <= 100):
        raise ValueError("Percentage p must be between 0 and 100.")

    # Calculate the number of elements to retain
    num_elements_to_retain = int(len(arr) * (100 - p) / 100)

    # If num_elements_to_retain is 0, return an empty array
    if num_elements_to_retain == 0:
        return np.array([])

    # Find the indices of the largest num_elements_to_retain elements
    indices_to_retain = np.argpartition(arr, -num_elements_to_retain)[-num_elements_to_retain:]

    # Return only the top (100-p)% of elements
    return arr[indices_to_retain]

In [5]:
"""
|MODEL_TYPE |  DATASET_TYPE    | NUMBER_OF_CLASSES| PARTITION      | ROUND_EPOCHS| SENSITIVITY_PERCENTAGE   | TRAIN/TEST_BATCH_SIZE | TRANSFORM_INPUT_SIZE  |
|-----------|------------------|------------------|----------------|-------------|--------------------------|-----------------------|-----------------------|
|cnn        |  fmnist          | 10               | noniid-#label2 | 1           | 10                       | 128                   | 128                   |
|resnet18   |  cifar10         | 10               | noniid-#label2 | 1           | 10                       | 128                   | 128                   |
|resnet50   |  cifar100        | 100              | noniid-#label20| 10          | 10                       | 128                   | 32                    |
|mobilenet  |  svhn            | 10               | noniid-#label2 | 1           | 10                       | 64                    | 224                   |
|alexnet    |  stl10           | 10               | noniid-#label10| 10          | 10                       | 128                    | 128                   |
|-----------|------------------|------------------|----------------|-------------|--------------------------|-----------------------|-----------------------|
"""
""" LEARNING HYPERPARAMETERS"""
MODEL_TYPE = "alexnet"
TRAIN_BATCH_SIZE=128
TEST_BATCH_SIZE=128
# by default set to 0.001 and for AlexNet set to 0.0001
LEARNING_RATE = 0.0001
WEIGHT_DECAY=1e-4
NUMBER_OF_CLIENTS = 10
# set to 10 for AlexNet
ROUND_EPOCHS = 6
""" DATA DISTRIBUTION """
DATASET_TYPE = "svhn"
PARTITION = "noniid-" + "fix"       # the second part accepted format is: "labeldir" (Dirichlet), "#label20", noniid-fix
DIRICHLET_BETA=0.1
NUMBER_OF_CLASSES = 10
TRANSFORM_INPUT_SIZE=128    # just works for svhn/stl10 dataset transformer
DESIRED_DISTRIBUTION = [
            [2948,    0, 5293,    0,    0,    0,    0,    0,    0,    0],
            [   0, 3466, 2330,    0,    0,    0,    0,    0,    0,    0],
            [2000,    0, 5292,    0,    0,    0,    0,    0,    0,    0],
            [   0,    0,    0, 4249, 3729,    0,    0,    0,    0,    0],
            [   0,    0,    0,    0, 3729,    0, 2465,    0,    0,    0],
            [   0,    0,    0, 3720,    0,    0, 2145,    0,    0,    0],
            [   0,    0,    0,    0,    0, 3865, 2864,    0,    0,    0],
            [   0,    0,    0,    0,    0,    0,    0, 1865, 2863,    0],
            [   0,    0,    0,    0,    0,    0,    0,    0, 5045, 3248],
            [   0,    0,    0,    0,    0,    0,    0, 3465,    0, 1329]
        ]

"""
SAFE-PFL
DISTANCE_METRIC values are:
- coordinate
- cosine
- euclidean
- jensen-shannon
- wasserstein
"""
SAVE_BEFORE_AGGREGATION_MODELS=True
DO_CLUSTER=True
CLUSTERING_PERIOD = 1           # Set to 1 to run simple Federated Learning with out clustering
FEDERATED_LEARNING_ROUNDS = 1   # The round in with Federated Learning will be executed
# set to 20 for ResNet50
SENSITIVITY_PERCENTAGE = 10
DISTANCE_METRIC = "coordinate"
# cosine similarity options
JUST_COMPARE_SIGNIFICANCE=False
ZERO_INSIGNIFICANT_IN_BOTH=False
COMPARE_MOST_SIGNIFICANCE_ONE=False
COMPARE_LESS_SIGNIFICANCE_ZERO=False   # Set to True for Selective Parameter Cosine when DISTANCE_METRIC is cosine

# Data


In [6]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

IMG_EXTENSIONS = (
    ".jpg",
    ".jpeg",
    ".png",
    ".ppm",
    ".bmp",
    ".pgm",
    ".tif",
    ".tiff",
    ".webp",
)


def mkdirs(dirpath):
    try:
        os.makedirs(dirpath)
    except Exception as _:
        pass


def accimage_loader(path):
    import accimage

    try:
        return accimage.Image(path)
    except IOError:
        return pil_loader(path)


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, "rb") as f:
        img = Image.open(f)
        return img.convert("RGB")


def default_loader(path):
    from torchvision import get_image_backend

    if get_image_backend() == "accimage":
        return accimage_loader(path)
    else:
        return pil_loader(path)


class CustomTensorDataset(data.TensorDataset):
    def __getitem__(self, index):
        return tuple(tensor[index] for tensor in self.tensors) + (index,)


class MNIST_truncated(data.Dataset):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):

        mnist_dataobj = MNIST(
            self.root, self.train, self.transform, self.target_transform, self.download
        )

        data = mnist_dataobj.data
        target = mnist_dataobj.targets

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]

        img = Image.fromarray(img.numpy(), mode="L")

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)


class FashionMNIST_truncated(data.Dataset):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):

        mnist_dataobj = FashionMNIST(
            self.root, self.train, self.transform, self.target_transform, self.download
        )

        data = mnist_dataobj.data
        target = mnist_dataobj.targets

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]

        img = Image.fromarray(img.numpy(), mode="L")

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)


class SVHN_custom(data.Dataset):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):
        if self.train is True:

            svhn_dataobj = SVHN(
                self.root, "train", self.transform, self.target_transform, self.download
            )
            data = svhn_dataobj.data
            target = svhn_dataobj.labels
        else:
            svhn_dataobj = SVHN(
                self.root, "test", self.transform, self.target_transform, self.download
            )
            data = svhn_dataobj.data
            target = svhn_dataobj.labels

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]
        return data, target

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(np.transpose(img, (1, 2, 0)))

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)


# torchvision CelebA
class CelebA_custom(VisionDataset):
    """`Large-scale CelebFaces Attributes (CelebA) Dataset <http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html>`_ Dataset.

    Args:
        root (string): Root directory where images are downloaded to.
        split (string): One of {'train', 'valid', 'test', 'all'}.
            Accordingly dataset is selected.
        target_type (string or list, optional): Type of target to use, ``attr``, ``identity``, ``bbox``,
            or ``landmarks``. Can also be a list to output a tuple with all specified target types.
            The targets represent:
                ``attr`` (np.array shape=(40,) dtype=int): binary (0, 1) labels for attributes
                ``identity`` (int): label for each person (data points with the same identity are the same person)
                ``bbox`` (np.array shape=(4,) dtype=int): bounding box (x, y, width, height)
                ``landmarks`` (np.array shape=(10,) dtype=int): landmark points (lefteye_x, lefteye_y, righteye_x,
                    righteye_y, nose_x, nose_y, leftmouth_x, leftmouth_y, rightmouth_x, rightmouth_y)
            Defaults to ``attr``. If empty, ``None`` will be returned as target.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """

    base_folder = "celeba"
    # There currently does not appear to be a easy way to extract 7z in python (without introducing additional
    # dependencies). The "in-the-wild" (not aligned+cropped) images are only in 7z, so they are not available
    # right now.
    file_list = [
        # File ID                         MD5 Hash                            Filename
        (
            "0B7EVK8r0v71pZjFTYXZWM3FlRnM",
            "00d2c5bc6d35e252742224ab0c1e8fcb",
            "img_align_celeba.zip",
        ),
        # ("0B7EVK8r0v71pbWNEUjJKdDQ3dGc", "b6cd7e93bc7a96c2dc33f819aa3ac651", "img_align_celeba_png.7z"),
        # ("0B7EVK8r0v71peklHb0pGdDl6R28", "b6cd7e93bc7a96c2dc33f819aa3ac651", "img_celeba.7z"),
        (
            "0B7EVK8r0v71pblRyaVFSWGxPY0U",
            "75e246fa4810816ffd6ee81facbd244c",
            "list_attr_celeba.txt",
        ),
        (
            "1_ee_0u7vcNLOfNLegJRHmolfH5ICW-XS",
            "32bd1bd63d3c78cd57e08160ec5ed1e2",
            "identity_CelebA.txt",
        ),
        (
            "0B7EVK8r0v71pbThiMVRxWXZ4dU0",
            "00566efa6fedff7a56946cd1c10f1c16",
            "list_bbox_celeba.txt",
        ),
        (
            "0B7EVK8r0v71pd0FJY3Blby1HUTQ",
            "cc24ecafdb5b50baae59b03474781f8c",
            "list_landmarks_align_celeba.txt",
        ),
        # ("0B7EVK8r0v71pTzJIdlJWdHczRlU", "063ee6ddb681f96bc9ca28c6febb9d1a", "list_landmarks_celeba.txt"),
        (
            "0B7EVK8r0v71pY0NSMzRuSXJEVkk",
            "d32c9cbf5e040fd4025c592c306e6668",
            "list_eval_partition.txt",
        ),
    ]

    def __init__(
        self,
        root,
        dataidxs=None,
        split="train",
        target_type="attr",
        transform=None,
        target_transform=None,
        download=False,
    ):
        import pandas

        super(CelebA_custom, self).__init__(
            root, transform=transform, target_transform=target_transform
        )
        self.split = split
        if isinstance(target_type, list):
            self.target_type = target_type
        else:
            self.target_type = [target_type]

        if not self.target_type and self.target_transform is not None:
            raise RuntimeError("target_transform is specified but target_type is empty")

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError(
                "Dataset not found or corrupted."
                + " You can use download=True to download it"
            )

        split_map = {
            "train": 0,
            "valid": 1,
            "test": 2,
            "all": None,
        }
        split = split_map[split.lower()]

        fn = partial(os.path.join, self.root, self.base_folder)
        splits = pandas.read_csv(
            fn("list_eval_partition.txt"),
            delim_whitespace=True,
            header=None,
            index_col=0,
        )
        identity = pandas.read_csv(
            fn("identity_CelebA.txt"), delim_whitespace=True, header=None, index_col=0
        )
        bbox = pandas.read_csv(
            fn("list_bbox_celeba.txt"), delim_whitespace=True, header=1, index_col=0
        )
        landmarks_align = pandas.read_csv(
            fn("list_landmarks_align_celeba.txt"), delim_whitespace=True, header=1
        )
        attr = pandas.read_csv(
            fn("list_attr_celeba.txt"), delim_whitespace=True, header=1
        )

        mask = slice(None) if split is None else (splits[1] == split)

        self.filename = splits[mask].index.values
        self.identity = torch.as_tensor(identity[mask].values)
        self.bbox = torch.as_tensor(bbox[mask].values)
        self.landmarks_align = torch.as_tensor(landmarks_align[mask].values)
        self.attr = torch.as_tensor(attr[mask].values)
        self.attr = (self.attr + 1) // 2  # map from {-1, 1} to {0, 1}
        self.attr_names = list(attr.columns)
        self.gender_index = self.attr_names.index("Male")
        self.dataidxs = dataidxs
        if self.dataidxs is None:
            self.target = self.attr[
                :, self.gender_index : self.gender_index + 1
            ].reshape(-1)
        else:
            self.target = self.attr[
                self.dataidxs, self.gender_index : self.gender_index + 1
            ].reshape(-1)

    def _check_integrity(self):
        for _, md5, filename in self.file_list:
            fpath = os.path.join(self.root, self.base_folder, filename)
            _, ext = os.path.splitext(filename)
            # Allow original archive to be deleted (zip and 7z)
            # Only need the extracted images
            if ext not in [".zip", ".7z"] and not check_integrity(fpath, md5):
                return False

        # Should check a hash of the images
        return os.path.isdir(
            os.path.join(self.root, self.base_folder, "img_align_celeba")
        )

    def download(self):
        import zipfile

        if self._check_integrity():
            print("Files already downloaded and verified")
            return

        for file_id, md5, filename in self.file_list:
            download_file_from_google_drive(
                file_id, os.path.join(self.root, self.base_folder), filename, md5
            )

        with zipfile.ZipFile(
            os.path.join(self.root, self.base_folder, "img_align_celeba.zip"), "r"
        ) as f:
            f.extractall(os.path.join(self.root, self.base_folder))

    def __getitem__(self, index):
        if self.dataidxs is None:
            X = PIL.Image.open(
                os.path.join(
                    self.root,
                    self.base_folder,
                    "img_align_celeba",
                    self.filename[index],
                )
            )

            target = []
            for t in self.target_type:
                if t == "attr":
                    target.append(self.attr[index, self.gender_index])
                elif t == "identity":
                    target.append(self.identity[index, 0])
                elif t == "bbox":
                    target.append(self.bbox[index, :])
                elif t == "landmarks":
                    target.append(self.landmarks_align[index, :])
                else:
                    # TODO: refactor with utils.verify_str_arg
                    raise ValueError('Target type "{}" is not recognized.'.format(t))
        else:
            X = PIL.Image.open(
                os.path.join(
                    self.root,
                    self.base_folder,
                    "img_align_celeba",
                    self.filename[self.dataidxs[index]],
                )
            )

            target = []
            for t in self.target_type:
                if t == "attr":
                    target.append(self.attr[self.dataidxs[index], self.gender_index])
                elif t == "identity":
                    target.append(self.identity[self.dataidxs[index], 0])
                elif t == "bbox":
                    target.append(self.bbox[self.dataidxs[index], :])
                elif t == "landmarks":
                    target.append(self.landmarks_align[self.dataidxs[index], :])
                else:
                    # TODO: refactor with utils.verify_str_arg
                    raise ValueError('Target type "{}" is not recognized.'.format(t))

        if self.transform is not None:
            X = self.transform(X)
        # print("target[0]:", target[0])
        if target:
            target = tuple(target) if len(target) > 1 else target[0]

            if self.target_transform is not None:
                target = self.target_transform(target)
        else:
            target = None
        # print("celeba target:", target)
        return X, target

    def __len__(self):
        if self.dataidxs is None:
            return len(self.attr)
        else:
            return len(self.dataidxs)

    def extra_repr(self):
        lines = ["Target type: {target_type}", "Split: {split}"]
        return "\n".join(lines).format(**self.__dict__)


class STL10_truncated(data.Dataset):
    def __init__(self, root, dataidxs=None, split="train", transform=None, target_transform=None, download=False):
        """
        Custom STL10 dataset with support for data indexing.
        Args:
            root (str): Dataset root directory.
            dataidxs (list, optional): Indices for data partitioning. Defaults to None.
            split (str, optional): Dataset split ('train', 'test', 'unlabeled'). Defaults to 'train'.
            transform (callable, optional): Transformations for the input data. Defaults to None.
            target_transform (callable, optional): Transformations for the target labels. Defaults to None.
            download (bool, optional): Whether to download the dataset. Defaults to False.
        """
        self.root = root
        self.dataidxs = dataidxs
        self.split = split
        self.transform = transform
        self.target_transform = target_transform
        self.download = download
        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):
        stl10_dataobj = STL10(
            self.root, split=self.split, transform=self.transform, target_transform=self.target_transform, download=self.download
        )
        data = stl10_dataobj.data
        target = np.array(stl10_dataobj.labels)

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is the class index.
        """
        img, target = self.data[index], self.target[index]

        # Ensure the image has the correct shape and dtype for PIL
        img = np.transpose(img, (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
        img = img.astype(np.uint8)          # Ensure dtype is uint8 for PIL compatibility
        img = Image.fromarray(img)          # Convert to PIL Image

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

class CIFAR10_truncated(data.Dataset):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):

        cifar_dataobj = CIFAR10(
            self.root, self.train, self.transform, self.target_transform, self.download
        )

        data = cifar_dataobj.data
        target = np.array(cifar_dataobj.targets)

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def truncate_channel(self, index):
        for i in range(index.shape[0]):
            gs_index = index[i]
            self.data[gs_index, :, :, 1] = 0.0
            self.data[gs_index, :, :, 2] = 0.0

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]

        # print("cifar10 img:", img)
        # print("cifar10 target:", target)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)


def gen_bar_updater() -> Callable[[int, int, int], None]:
    pbar = tqdm(total=None)

    def bar_update(count, block_size, total_size):
        if pbar.total is None and total_size:
            pbar.total = total_size
        progress_bytes = count * block_size
        pbar.update(progress_bytes - pbar.n)

    return bar_update


def download_url(
    url: str, root: str, filename: Optional[str] = None, md5: Optional[str] = None
) -> None:
    """Download a file from a url and place it in root.
    Args:
        url (str): URL to download file from
        root (str): Directory to place downloaded file in
        filename (str, optional): Name to save the file under. If None, use the basename of the URL
        md5 (str, optional): MD5 checksum of the download. If None, do not check
    """
    import urllib

    root = os.path.expanduser(root)
    if not filename:
        filename = os.path.basename(url)
    fpath = os.path.join(root, filename)

    os.makedirs(root, exist_ok=True)

    # check if file is already present locally
    if check_integrity(fpath, md5):
        print("Using downloaded and verified file: " + fpath)
    else:  # download the file
        try:
            print("Downloading " + url + " to " + fpath)
            urllib.request.urlretrieve(url, fpath, reporthook=gen_bar_updater())
        except (urllib.error.URLError, IOError) as e:  # type: ignore[attr-defined]
            if url[:5] == "https":
                url = url.replace("https:", "http:")
                print(
                    "Failed download. Trying https -> http instead."
                    " Downloading " + url + " to " + fpath
                )
                urllib.request.urlretrieve(url, fpath, reporthook=gen_bar_updater())
            else:
                raise e
        # check integrity of downloaded file
        if not check_integrity(fpath, md5):
            raise RuntimeError("File not found or corrupted.")


def _is_tarxz(filename: str) -> bool:
    return filename.endswith(".tar.xz")


def _is_tar(filename: str) -> bool:
    return filename.endswith(".tar")


def _is_targz(filename: str) -> bool:
    return filename.endswith(".tar.gz")


def _is_tgz(filename: str) -> bool:
    return filename.endswith(".tgz")


def _is_gzip(filename: str) -> bool:
    return filename.endswith(".gz") and not filename.endswith(".tar.gz")


def _is_zip(filename: str) -> bool:
    return filename.endswith(".zip")


def extract_archive(
    from_path: str, to_path: Optional[str] = None, remove_finished: bool = False
) -> None:
    if to_path is None:
        to_path = os.path.dirname(from_path)

    if _is_tar(from_path):
        with tarfile.open(from_path, "r") as tar:

            def is_within_directory(directory, target):

                abs_directory = os.path.abspath(directory)
                abs_target = os.path.abspath(target)

                prefix = os.path.commonprefix([abs_directory, abs_target])

                return prefix == abs_directory

            def safe_extract(tar, path=".", members=None, *, numeric_owner=False):

                for member in tar.getmembers():
                    member_path = os.path.join(path, member.name)
                    if not is_within_directory(path, member_path):
                        raise Exception("Attempted Path Traversal in Tar File")

                tar.extractall(path, members, numeric_owner=numeric_owner)

            safe_extract(tar, path=to_path)
    elif _is_targz(from_path) or _is_tgz(from_path):
        with tarfile.open(from_path, "r:gz") as tar:

            def is_within_directory(directory, target):

                abs_directory = os.path.abspath(directory)
                abs_target = os.path.abspath(target)

                prefix = os.path.commonprefix([abs_directory, abs_target])

                return prefix == abs_directory

            def safe_extract(tar, path=".", members=None, *, numeric_owner=False):

                for member in tar.getmembers():
                    member_path = os.path.join(path, member.name)
                    if not is_within_directory(path, member_path):
                        raise Exception("Attempted Path Traversal in Tar File")

                tar.extractall(path, members, numeric_owner=numeric_owner)

            safe_extract(tar, path=to_path)
    elif _is_tarxz(from_path):
        with tarfile.open(from_path, "r:xz") as tar:

            def is_within_directory(directory, target):

                abs_directory = os.path.abspath(directory)
                abs_target = os.path.abspath(target)

                prefix = os.path.commonprefix([abs_directory, abs_target])

                return prefix == abs_directory

            def safe_extract(tar, path=".", members=None, *, numeric_owner=False):

                for member in tar.getmembers():
                    member_path = os.path.join(path, member.name)
                    if not is_within_directory(path, member_path):
                        raise Exception("Attempted Path Traversal in Tar File")

                tar.extractall(path, members, numeric_owner=numeric_owner)

            safe_extract(tar, path=to_path)
    elif _is_gzip(from_path):
        to_path = os.path.join(
            to_path, os.path.splitext(os.path.basename(from_path))[0]
        )
        with open(to_path, "wb") as out_f, gzip.GzipFile(from_path) as zip_f:
            out_f.write(zip_f.read())
    elif _is_zip(from_path):
        with zipfile.ZipFile(from_path, "r") as z:
            z.extractall(to_path)
    else:
        raise ValueError("Extraction of {} not supported".format(from_path))

    if remove_finished:
        os.remove(from_path)


def download_and_extract_archive(
    url: str,
    download_root: str,
    extract_root: Optional[str] = None,
    filename: Optional[str] = None,
    md5: Optional[str] = None,
    remove_finished: bool = False,
) -> None:
    download_root = os.path.expanduser(download_root)
    if extract_root is None:
        extract_root = download_root
    if not filename:
        filename = os.path.basename(url)

    download_url(url, download_root, filename, md5)

    archive = os.path.join(download_root, filename)
    print("Extracting {} to {}".format(archive, extract_root))
    extract_archive(archive, extract_root, remove_finished)


class FEMNIST(MNIST):
    """
    This dataset is derived from the Leaf repository
    (https://github.com/TalwalkarLab/leaf) pre-processing of the Extended MNIST
    dataset, grouping examples by writer. Details about Leaf were published in
    "LEAF: A Benchmark for Federated Settings" https://arxiv.org/abs/1812.01097.
    """

    resources = [
        (
            "https://raw.githubusercontent.com/tao-shen/FEMNIST_pytorch/master/femnist.tar.gz",
            "59c65cec646fc57fe92d27d83afdf0ed",
        )
    ]

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):
        super(MNIST, self).__init__(
            root, transform=transform, target_transform=target_transform
        )
        self.train = train
        self.dataidxs = dataidxs

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError(
                "Dataset not found." + " You can use download=True to download it"
            )
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file

        self.data, self.targets, self.users_index = torch.load(
            os.path.join(self.processed_folder, data_file)
        )

        if self.dataidxs is not None:
            self.data = self.data[self.dataidxs]
            self.targets = self.targets[self.dataidxs]

    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index])
        img = Image.fromarray(img.numpy(), mode="F")
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def download(self):
        """Download the FEMNIST data if it doesn't exist in processed_folder already."""
        import shutil

        if self._check_exists():
            return

        mkdirs(self.raw_folder)
        mkdirs(self.processed_folder)

        # download files
        for url, md5 in self.resources:
            filename = url.rpartition("/")[2]
            download_and_extract_archive(
                url, download_root=self.raw_folder, filename=filename, md5=md5
            )

        # process and save as torch files
        print("Processing...")
        shutil.move(
            os.path.join(self.raw_folder, self.training_file), self.processed_folder
        )
        shutil.move(
            os.path.join(self.raw_folder, self.test_file), self.processed_folder
        )

    def __len__(self):
        return len(self.data)

    def _check_exists(self) -> bool:
        return all(
            check_integrity(
                os.path.join(
                    self.raw_folder,
                    os.path.splitext(os.path.basename(url))[0]
                    + os.path.splitext(os.path.basename(url))[1],
                )
            )
            for url, _ in self.resources
        )


class Generated(MNIST):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):
        super(MNIST, self).__init__(
            root, transform=transform, target_transform=target_transform
        )
        self.train = train
        self.dataidxs = dataidxs

        if self.train:
            self.data = np.load("data/generated/X_train.npy")
            self.targets = np.load("data/generated/y_train.npy")
        else:
            self.data = np.load("data/generated/X_test.npy")
            self.targets = np.load("data/generated/y_test.npy")

        if self.dataidxs is not None:
            self.data = self.data[self.dataidxs]
            self.targets = self.targets[self.dataidxs]

    def __getitem__(self, index):
        data, target = self.data[index], self.targets[index]
        return data, target

    def __len__(self):
        return len(self.data)


class genData(MNIST):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        data, target = self.data[index], self.targets[index]
        return data, target

    def __len__(self):
        return len(self.data)


class CIFAR100_truncated(data.Dataset):

    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=False,
    ):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):

        cifar_dataobj = CIFAR100(
            self.root, self.train, self.transform, self.target_transform, self.download
        )

        if torchvision.__version__ == "0.2.1":
            if self.train:
                data, target = cifar_dataobj.train_data, np.array(
                    cifar_dataobj.train_labels
                )
            else:
                data, target = cifar_dataobj.test_data, np.array(
                    cifar_dataobj.test_labels
                )
        else:
            data = cifar_dataobj.data
            target = np.array(cifar_dataobj.targets)

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]
        img = Image.fromarray(img)
        # print("cifar10 img:", img)
        # print("cifar10 target:", target)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)


class ImageFolder_custom(DatasetFolder):
    def __init__(
        self,
        root,
        dataidxs=None,
        train=True,
        transform=None,
        target_transform=None,
        download=None,
    ):
        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        imagefolder_obj = ImageFolder(self.root, self.transform, self.target_transform)
        self.loader = imagefolder_obj.loader
        if self.dataidxs is not None:
            self.samples = np.array(imagefolder_obj.samples)[self.dataidxs]
        else:
            self.samples = np.array(imagefolder_obj.samples)

    def __getitem__(self, index):
        path = self.samples[index][0]
        target = self.samples[index][1]
        target = int(target)
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    def __len__(self):
        if self.dataidxs is None:
            return len(self.samples)
        else:
            return len(self.dataidxs)

In [7]:
def mkdirs(dirpath):
    try:
        os.makedirs(dirpath)
    except Exception as _:
        pass


def load_mnist_data(datadir):
    transform = transforms.Compose([transforms.ToTensor()])
    mnist_train_ds = MNIST_truncated(
        datadir, train=True, download=True, transform=transform
    )
    mnist_test_ds = MNIST_truncated(
        datadir, train=False, download=True, transform=transform
    )
    X_train, y_train = mnist_train_ds.data, mnist_train_ds.target
    X_test, y_test = mnist_test_ds.data, mnist_test_ds.target
    X_train = X_train.data.numpy()
    y_train = y_train.data.numpy()
    X_test = X_test.data.numpy()
    y_test = y_test.data.numpy()
    return (X_train, y_train, X_test, y_test)


def load_fmnist_data(datadir):
    transform = transforms.Compose([transforms.ToTensor()])
    mnist_train_ds = FashionMNIST_truncated(
        datadir, train=True, download=True, transform=transform
    )
    mnist_test_ds = FashionMNIST_truncated(
        datadir, train=False, download=True, transform=transform
    )
    X_train, y_train = mnist_train_ds.data, mnist_train_ds.target
    X_test, y_test = mnist_test_ds.data, mnist_test_ds.target
    X_train = X_train.data.numpy()
    y_train = y_train.data.numpy()
    X_test = X_test.data.numpy()
    y_test = y_test.data.numpy()
    return (X_train, y_train, X_test, y_test)


def load_svhn_data(datadir):
    transform = transforms.Compose([
        transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    svhn_train_ds = SVHN_custom(datadir, train=True, download=True, transform=transform)
    svhn_test_ds = SVHN_custom(datadir, train=False, download=True, transform=transform)
    X_train, y_train = svhn_train_ds.data, svhn_train_ds.target
    X_test, y_test = svhn_test_ds.data, svhn_test_ds.target
    # X_train = X_train.data.numpy()
    # y_train = y_train.data.numpy()
    # X_test = X_test.data.numpy()
    # y_test = y_test.data.numpy()
    return (X_train, y_train, X_test, y_test)


def load_cifar10_data(datadir):
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    cifar10_train_ds = CIFAR10_truncated(
        datadir, train=True, download=True, transform=transform
    )
    cifar10_test_ds = CIFAR10_truncated(
        datadir, train=False, download=True, transform=transform
    )
    X_train, y_train = cifar10_train_ds.data, cifar10_train_ds.target
    X_test, y_test = cifar10_test_ds.data, cifar10_test_ds.target

    return (X_train, y_train, X_test, y_test)


def load_celeba_data(datadir):
    transform = transforms.Compose([transforms.ToTensor()])
    celeba_train_ds = CelebA_custom(
        datadir, split="train", target_type="attr", download=True, transform=transform
    )
    celeba_test_ds = CelebA_custom(
        datadir, split="test", target_type="attr", download=True, transform=transform
    )
    gender_index = celeba_train_ds.attr_names.index("Male")
    y_train = celeba_train_ds.attr[:, gender_index : gender_index + 1].reshape(-1)
    y_test = celeba_test_ds.attr[:, gender_index : gender_index + 1].reshape(-1)
    # y_train = y_train.numpy()
    # y_test = y_test.numpy()
    return (None, y_train, None, y_test)


def load_femnist_data(datadir):
    transform = transforms.Compose([transforms.ToTensor()])
    mnist_train_ds = FEMNIST(datadir, train=True, transform=transform, download=True)
    mnist_test_ds = FEMNIST(datadir, train=False, transform=transform, download=True)
    X_train, y_train, u_train = (
        mnist_train_ds.data,
        mnist_train_ds.targets,
        mnist_train_ds.users_index,
    )
    X_test, y_test, u_test = (
        mnist_test_ds.data,
        mnist_test_ds.targets,
        mnist_test_ds.users_index,
    )
    X_train = X_train.data.numpy()
    y_train = y_train.data.numpy()
    u_train = np.array(u_train)
    X_test = X_test.data.numpy()
    y_test = y_test.data.numpy()
    u_test = np.array(u_test)
    return (X_train, y_train, u_train, X_test, y_test, u_test)


def load_cifar100_data(datadir):
    transform = transforms.Compose([transforms.ToTensor()])
    cifar100_train_ds = CIFAR100_truncated(
        datadir, train=True, download=True, transform=transform
    )
    cifar100_test_ds = CIFAR100_truncated(
        datadir, train=False, download=True, transform=transform
    )
    X_train, y_train = cifar100_train_ds.data, cifar100_train_ds.target
    X_test, y_test = cifar100_test_ds.data, cifar100_test_ds.target
    # y_train = y_train.numpy()
    # y_test = y_test.numpy()
    return (X_train, y_train, X_test, y_test)


def load_tinyimagenet_data(datadir):
    split = "val"
    TinyImageNet(datadir, split=split)
    transform_train = transforms.Compose([
        transforms.RandomCrop(64, padding=4),  # Random cropping with padding
        transforms.RandomHorizontalFlip(),  # Horizontal flip
        transforms.RandomRotation(15),  # Random rotation
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262]),  # Normalization
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262]),
    ])
    # transform = transforms.Compose([transforms.ToTensor()])
    xray_train_ds = ImageFolder_custom(
        datadir + "tiny-imagenet-200/train/", transform=transform_train
    )
    xray_test_ds = ImageFolder_custom(
        datadir + "tiny-imagenet-200/val/", transform=transform_test
    )
    X_train, y_train = np.array([s[0] for s in xray_train_ds.samples]), np.array(
        [int(s[1]) for s in xray_train_ds.samples]
    )
    X_test, y_test = np.array([s[0] for s in xray_test_ds.samples]), np.array(
        [int(s[1]) for s in xray_test_ds.samples]
    )
    return (X_train, y_train, X_test, y_test)

def load_stl10_data(datadir):
    transform_train = transforms.Compose([
    transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    transform_test = transforms.Compose([
    transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    stl10_train_ds = STL10_truncated(datadir, split="train", transform=transform_train, download=True)
    stl10_test_ds = STL10_truncated(datadir, split="test", transform=transform_test, download=True)

    X_train, y_train = stl10_train_ds.data, stl10_train_ds.target
    X_test, y_test = stl10_test_ds.data, stl10_test_ds.target

    return X_train, y_train, X_test, y_test

def record_net_data_stats(y_train, net_dataidx_map, logdir):
    net_cls_counts = {}
    for net_i, dataidx in net_dataidx_map.items():
        unq, unq_cnt = np.unique(y_train[dataidx], return_counts=True)
        tmp = {unq[i]: unq_cnt[i] for i in range(len(unq))}
        net_cls_counts[net_i] = tmp
    logger.info("Data statistics: %s" % str(net_cls_counts))
    return net_cls_counts

In [8]:
def partition_data(dataset, datadir, logdir, partition, n_parties, beta=0.1):
    # Optional: set random seeds for reproducibility
    np.random.seed(2020)
    torch.manual_seed(2020)

    # Initialize test data index map
    test_dataidx_map = {}

    # Load dataset
    if dataset == "mnist":
        X_train, y_train, X_test, y_test = load_mnist_data(datadir)
    elif dataset == "fmnist":
        X_train, y_train, X_test, y_test = load_fmnist_data(datadir)
    elif dataset == "cifar10":
        X_train, y_train, X_test, y_test = load_cifar10_data(datadir)
    elif dataset == "svhn":
        X_train, y_train, X_test, y_test = load_svhn_data(datadir)
    elif dataset == "celeba":
        X_train, y_train, X_test, y_test = load_celeba_data(datadir)
    elif dataset == "femnist":
        X_train, y_train, u_train, X_test, y_test, u_test = load_femnist_data(datadir)
    elif dataset == "cifar100":
        X_train, y_train, X_test, y_test = load_cifar100_data(datadir)
    elif dataset == "tinyimagenet":
        X_train, y_train, X_test, y_test = load_tinyimagenet_data(datadir)
    elif dataset == "stl10":
        X_train, y_train, X_test, y_test = load_stl10_data(datadir)
    elif dataset == "generated":
        # Code for generated dataset (omitted for brevity)
        pass
    # Add other datasets if needed

    n_train = y_train.shape[0]

    # Partition the data
    if partition == "homo":
        # Homogeneous data partition
        idxs = np.random.permutation(n_train)
        batch_idxs = np.array_split(idxs, n_parties)
        net_dataidx_map = {i: batch_idxs[i] for i in range(n_parties)}

    elif partition == "noniid-labeldir":
        # Existing non-IID logic using Dirichlet distribution
        min_size = 0
        min_require_size = 10  # Minimum number required for each party
        K = 10  # Number of classes

        N = y_train.shape[0]
        net_dataidx_map = {}
        test_dataidx_map = {}  # Make sure to initialize this

        while min_size < min_require_size:
            idx_batch = [[] for _ in range(n_parties)]
            for k in range(K):
                idx_k = np.where(y_train == k)[0]
                np.random.shuffle(idx_k)
                proportions = np.random.dirichlet(np.repeat(beta, n_parties))
                proportions = np.array([p * (len(idx_j) < N / n_parties) for p, idx_j in zip(proportions, idx_batch)])
                proportions = proportions / proportions.sum()  # Normalize
                proportions = (np.cumsum(proportions) * len(idx_k)).astype(int)[:-1]
                idx_batch = [idx_j + idx.tolist() for idx_j, idx in zip(idx_batch, np.split(idx_k, proportions))]

            min_size = min([len(idx_j) for idx_j in idx_batch])

        for j in range(n_parties):
            np.random.shuffle(idx_batch[j])
            net_dataidx_map[j] = idx_batch[j]

            # Initialize test_dataidx_map for current party
            test_dataidx_map[j] = []

            # Gather test indices for current party based on labels in net_dataidx_map[j]
            for k in range(K):
                if k in y_train[net_dataidx_map[j]]:
                    # Access test indices for class k
                    idx_test_k = np.where(y_test == k)[0]
                    np.random.shuffle(idx_test_k)

                    # The number of sample for each party based on training set size
                    n_samples = int(len(net_dataidx_map[j]) * len(idx_test_k) / N)
                    test_dataidx_map[j].extend(idx_test_k[:n_samples])

            test_dataidx_map[j] = np.array(test_dataidx_map[j])

        # Cleanup to avoid empty concatenation error
        for j in range(n_parties):
            if len(test_dataidx_map[j]) == 0:
                test_dataidx_map[j] = np.array([])  # Set to an empty array to avoid errors later

    elif partition == "noniid-fix":
        # Custom fixed distribution logic
        desired_distribution = DESIRED_DISTRIBUTION

        # Number of clients and classes
        num_clients = len(desired_distribution)
        num_classes = len(desired_distribution[0])

        assert num_clients == NUMBER_OF_CLIENTS
        assert num_classes == NUMBER_OF_CLASSES

        # Initialize the data indices for each client
        net_dataidx_map = {i: [] for i in range(num_clients)}

        # Iterate over each class and assign samples to clients based on the desired distribution
        for class_idx in range(num_classes):
            # Get the indices of all samples belonging to the current class
            class_indices = np.where(y_train == class_idx)[0]

            # Shuffle the indices to ensure randomness
            np.random.shuffle(class_indices)

            # Assign samples to clients based on the desired distribution
            start_idx = 0
            for client_idx in range(num_clients):
                num_samples = desired_distribution[client_idx][class_idx]
                if num_samples > 0:
                    end_idx = start_idx + num_samples
                    net_dataidx_map[client_idx].extend(class_indices[start_idx:end_idx])
                    start_idx = end_idx

        # Initialize test_dataidx_map for each client
        for j in range(num_clients):
            test_dataidx_map[j] = []

            # Gather test indices for current party based on labels in net_dataidx_map[j]
            for k in range(num_classes):
                if k in y_train[net_dataidx_map[j]]:
                    # Access test indices for class k
                    idx_test_k = np.where(y_test == k)[0]
                    np.random.shuffle(idx_test_k)

                    # The number of samples for each party based on training set size
                    n_samples = int(len(net_dataidx_map[j]) * len(idx_test_k) / n_train)
                    test_dataidx_map[j].extend(idx_test_k[:n_samples])

            test_dataidx_map[j] = np.array(test_dataidx_map[j])

        # Cleanup to avoid empty concatenation error
        for j in range(num_clients):
            if len(test_dataidx_map[j]) == 0:
                test_dataidx_map[j] = np.array([])  # Set to an empty array to avoid errors later

    elif partition.startswith("noniid-#label") and partition[13:].isdigit():
        # Existing logic for noniid-#label partitioning
        num = int(partition[13:])
        if dataset in ("celeba", "covtype", "a9a", "rcv1", "SUSY"):
            num = 1
            K = 2
        else:
            if dataset == "cifar100":
                K = 100
            elif dataset == "tinyimagenet":
                K = 200
            else:
                K = 10
        if num == K:
            # IID partition
            net_dataidx_map = {
                i: np.ndarray(0, dtype=np.int64) for i in range(n_parties)
            }
            for i in range(K):
                idx_k = np.where(y_train == i)[0]
                np.random.shuffle(idx_k)
                split = np.array_split(idx_k, n_parties)
                for j in range(n_parties):
                    net_dataidx_map[j] = np.append(net_dataidx_map[j], split[j])
        else:
            times = [0 for _ in range(K)]
            contain = []
            for i in range(n_parties):
                current = [i % K]
                times[i % K] += 1
                j = 1
                while j < num:
                    ind = random.randint(0, K - 1)
                    if ind not in current:
                        j += 1
                        current.append(ind)
                        times[ind] += 1
                contain.append(current)
            net_dataidx_map = {
                i: np.ndarray(0, dtype=np.int64) for i in range(n_parties)
            }
            test_dataidx_map = {
                i: np.ndarray(0, dtype=np.int64) for i in range(n_parties)
            }
            for i in range(K):
                if times[i] > 0:
                    idx_k = np.where(y_train == i)[0]
                    idx_t = np.where(y_test == i)[0]
                    np.random.shuffle(idx_k)
                    np.random.shuffle(idx_t)
                    split = np.array_split(idx_k, times[i])
                    splitt = np.array_split(idx_t, times[i])
                    ids = 0
                    for j in range(n_parties):
                        if i in contain[j]:
                            net_dataidx_map[j] = np.append(
                                net_dataidx_map[j], split[ids]
                            )
                            test_dataidx_map[j] = np.append(
                                test_dataidx_map[j], splitt[ids]
                            )
                            ids += 1
    else:
        raise ValueError(f"Unknown partition method: {partition}")

    # Record the data statistics
    traindata_cls_counts = record_net_data_stats(y_train, net_dataidx_map, logdir)

    return (
        X_train,
        y_train,
        X_test,
        y_test,
        net_dataidx_map,
        test_dataidx_map,
        traindata_cls_counts,
    )

In [9]:
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0, net_id=None, total=0):
        self.std = std
        self.mean = mean
        self.net_id = net_id
        self.num = int(sqrt(total))
        if self.num * self.num < total:
            self.num = self.num + 1

    def __call__(self, tensor):
        if self.net_id is None:
            return tensor + torch.randn(tensor.size()) * self.std + self.mean
        else:
            tmp = torch.randn(tensor.size())
            filt = torch.zeros(tensor.size())
            size = int(28 / self.num)
            row = int(self.net_id / size)
            col = self.net_id % size
            for i in range(size):
                for j in range(size):
                    filt[:, row * size + i, col * size + j] = 1
            tmp = tmp * filt
            return tensor + tmp * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + "(mean={0}, std={1})".format(
            self.mean, self.std
        )


def get_dataloader(
    dataset,
    datadir,
    train_bs,
    test_bs,
    dataidxs=None,
    testidxs=None,
    noise_level=0,
    net_id=None,
    total=0,
):
    if dataset in (
        "mnist",
        "femnist",
        "fmnist",
        "cifar10",
        "svhn",
        "generated",
        "covtype",
        "a9a",
        "rcv1",
        "SUSY",
        "cifar100",
        "tinyimagenet",
        "stl10"
    ):
        if dataset == "mnist":
            dl_obj = MNIST_truncated
            transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
            transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
        elif dataset == "femnist":
            dl_obj = FEMNIST
            transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
            transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
        elif dataset == "fmnist":
            dl_obj = FashionMNIST_truncated
            transform_train = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
            transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
        elif dataset == "svhn":
            dl_obj = SVHN_custom
            transform_train = transforms.Compose([
                transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])
            transform_test = transforms.Compose([
                transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])

        elif dataset == "cifar10":
            print("in cifar10")
            dl_obj = CIFAR10_truncated
            transform_train = transforms.Compose(
                [
                    # transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    transforms.Lambda(
                        lambda x: F.pad(
                            Variable(x.unsqueeze(0), requires_grad=False),
                            (4, 4, 4, 4),
                            mode="reflect",
                        ).data.squeeze()
                    ),
                    transforms.ToPILImage(),
                    transforms.RandomCrop(32),
                    transforms.ToTensor(),
                    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
            # data prep for test set
            transform_test = transforms.Compose(
                [
                    transforms.ToTensor(),
                    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                    AddGaussianNoise(0.0, noise_level, net_id, total),
                ]
            )
        elif dataset == "cifar100":
            print("in 100")
            dl_obj = CIFAR100_truncated
            normalize = transforms.Normalize(
                mean=[0.5070751592371323, 0.48654887331495095, 0.4409178433670343],
                std=[0.2673342858792401, 0.2564384629170883, 0.27615047132568404],
            )

            transform_train = transforms.Compose(
                [
                    # transforms.ToPILImage(),
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(15),
                    transforms.ToTensor(),
                    normalize,
                ]
            )
            # data prep for test set
            transform_test = transforms.Compose([transforms.ToTensor(), normalize])
        elif dataset == "tinyimagenet":
            dl_obj = ImageFolder_custom
            transform_train = transforms.Compose([
                transforms.RandomCrop(64, padding=4),  # Random cropping with padding
                transforms.RandomHorizontalFlip(),  # Horizontal flip
                transforms.RandomRotation(15),  # Random rotation
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262]),  # Normalization
            ])

            transform_test = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262]),
            ])
        elif dataset == "stl10":
            dl_obj = STL10_truncated
            transform_train = transforms.Compose([
                transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
            transform_test = transforms.Compose([
                transforms.Resize((TRANSFORM_INPUT_SIZE, TRANSFORM_INPUT_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            dl_obj = Generated
            transform_train = None
            transform_test = None
        if dataset == "tinyimagenet":
            train_ds = dl_obj(
                datadir + "tiny-imagenet-200/train/",
                dataidxs=dataidxs,
                transform=transform_train,
            )
            test_ds = dl_obj(
                datadir + "tiny-imagenet-200/val/",
                dataidxs=testidxs,
                transform=transform_test
            )
        elif dataset == "stl10":
            train_ds = dl_obj(
                datadir,
                dataidxs=dataidxs,
                split="train",
                transform=transform_train,
                download=True
            )
            test_ds = dl_obj(
                datadir,
                dataidxs=testidxs,
                split="test",
                transform=transform_test,
                download=True
            )
        else:
            print("dir", datadir)
            train_ds = dl_obj(
                datadir,
                dataidxs=dataidxs,
                train=True,
                transform=transform_train,
                download=True,
            )
            test_ds = dl_obj(
                datadir,
                dataidxs=testidxs,
                train=False,
                transform=transform_test,
                download=True,
            )
        train_dl = data.DataLoader(
            dataset=train_ds, batch_size=train_bs, shuffle=True, drop_last=False
        )
        test_dl = data.DataLoader(
            dataset=test_ds, batch_size=test_bs, shuffle=False, drop_last=False
        )
        print(train_ds, "train ds")
    return train_dl, test_dl, train_ds, test_ds

In [10]:
def get_loaders(NUMBER_OF_CLIENTS):

    (
        X_train,
        y_train,
        X_test,
        y_test,
        net_dataidx_map,
        test_dataidx_map,
        traindata_cls_counts,
    ) = partition_data(
        dataset=DATASET_TYPE,
        datadir="./data/",
        logdir="./logs/",
        partition=PARTITION,
        n_parties=10,
    )
    print("shapes", X_train.shape, y_train.shape)
    train_loaders = []
    test_loaders = []
    for client_id in range(NUMBER_OF_CLIENTS):

        dataidxs = net_dataidx_map[client_id]
        testidxs = test_dataidx_map[client_id]

        train_dl_local, test_dl_local, train_ds_local, test_ds_local = get_dataloader(
            dataset=DATASET_TYPE,
            datadir="./data/",
            train_bs=TRAIN_BATCH_SIZE,
            test_bs=TEST_BATCH_SIZE,
            dataidxs=dataidxs,
            testidxs=testidxs,
        )
        train_loaders.append(train_dl_local)
        test_loaders.append(test_dl_local)

    return train_loaders, test_loaders

In [11]:
def load_and_prepare_data():
    train_loaders, test_loaders = get_loaders(10)
    return train_loaders, test_loaders

In [12]:
train_loaders, test_loaders = load_and_prepare_data()

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO:root:Data statistics: {0: {0: 2948, 2: 5293}, 1: {1: 3466, 2: 2330}, 2: {0: 2000, 2: 2962}, 3: {3: 4249, 4: 3729}, 4: {4: 3729, 6: 2465}, 5: {3: 3720, 6: 2145}, 6: {5: 3865, 6: 1117}, 7: {7: 1865, 8: 2863}, 8: {8: 2182, 9: 3248}, 9: {7: 3465, 9: 1329}}


shapes (73257, 3, 32, 32) (73257,)
dir ./data/
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
<__main__.SVHN_custom object at 0x7a10f8a35120> train ds
dir ./data/
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
<__main__.SVHN_custom object at 0x7a101df530a0> train ds
dir ./data/
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
<__main__.SVHN_custom object at 0x7a101df52e60> train ds
dir ./data/
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
<__main__.SVHN_custom object at 0x7a101df531f0> train ds
dir ./data/
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
<__main__.SVHN_custom object at 0x7a101df51990> train ds
dir ./data/
Using downloaded and ve

In [13]:
import gdown

gdown.download('https://drive.google.com/file/d/1iQ_rUFeSYerCM_4O4EkgnPlw2o9kHKcI/view?usp=drive_link', './node_0.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1Fig4VlVQqakq7LgJ_inbciLt6qIWqhuE/view?usp=drive_link', './node_1.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1UDBswYJXbd6aimlQek-qsiPTRD7H-Gmw/view?usp=drive_link', './node_2.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1Ux3Xx5o9RwPhpE6ikkot6APnUEg1hzTC/view?usp=drive_link', './node_3.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1VDfqqI2JyGVXXjwhz7JEd2eGs1sym78y/view?usp=drive_link', './node_4.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1rSaoong2Wrtl55f1ewwoyE4z-JOX-7M-/view?usp=drive_link', './node_5.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1nk_T2_iaO0jJkjorvxnRC1SkBpCvMAbt/view?usp=drive_link', './node_6.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1xUJ8C7k_--U56pVAnzcI5lofNMuqukh0/view?usp=drive_link', './node_7.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/18ywhNBRkesxrL2UMEn0iuE6Kl5DEE4Nu/view?usp=drive_link', './node_8.pth', quiet=True,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1AfY9qtHRhUG4PX9F4_kjhLS29DQ173ss/view?usp=drive_link', './node_9.pth', quiet=True,fuzzy=True)

'./node_9.pth'

# Net

In [14]:
class Net(nn.Module):
    def __init__(self):  # Corrected: usage of double underscores
        super(Net, self).__init__()  # Call the parent class constructor
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Calculate the size of the output from the last convolutional layer
        self._to_linear = 128 * (128 // 8) * (128 // 8)  # Adjust based on pooling layers
        self.classifier = nn.Sequential(
            nn.Linear(self._to_linear, 512),  # Adjusted input size
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, NUMBER_OF_CLASSES),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

# Clustering


In [15]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def find_num_cluster(clusters):
    return len(set(clusters))

def load_torch_model(node_id):
    checkpoint_path = f"./node_{node_id}.pth"
    state_dict = torch.load(checkpoint_path)
    model = Net()
    model.load_state_dict(state_dict, strict=False)
    return model

class Clustering:
    def __init__(self, clients, trainLoaders, percentage):
        self.clients = clients
        self.num_nodes = len(clients)
        self.percentage = percentage
        self.Mask_Number = 0
        self.maskIds = []
        self.grads = []
        self.load_and_calculate_sensitivity(trainLoaders)
        self.distances = self.calculate_distance()
        self.Clusters = self.make_clusters()

    def assign_save_ids_to_weights(self, model):
        weight_id_map = {}
        weight_id = 0
        for name, parameter in model.named_parameters():
            weight_id_map[name] = {}
            num_weights = parameter.numel()
            for i in range(num_weights):
                weight_id_map[name][i] = weight_id
                weight_id += 1
        filename = "weight_to_id.csv"
        if not os.path.exists(filename):
            with open(filename, "w", newline="") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["Layer", "Weight Index", "Weight ID"])
                for layer_name, indices in weight_id_map.items():
                    for index, weight_id in indices.items():
                        writer.writerow([layer_name, index, weight_id])
        return weight_id_map

    def load_and_calculate_sensitivity(self, trainLoaders):
        """
        Calculate sensitivity for each client and store the results in the object.
        """
        for cid in self.clients:
            model = load_torch_model(cid).to(DEVICE)
            model.eval()
            sensitivity_value = self.calculate_sensitivity(model, trainLoaders[int(cid)])
            weight_id_map = self.assign_save_ids_to_weights(model)

            mask_ID, weights = self.get_maskIds(sensitivity_value, weight_id_map, self.percentage)
            print(f"Model weights and sensitivity data for client #{cid} processed.")

            self.maskIds.append(mask_ID)
            self.grads.append(weights)

    def calculate_sensitivity(self, model, dataloader):
        model.eval()
        criterion = nn.CrossEntropyLoss()
        gradient_sums = {}
        for name, param in model.named_parameters():
            gradient_sums[name] = 0.0
            param.requires_grad_(True)
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            model.zero_grad()
            loss.backward()
            sensitivities = {}
            for name, parameter in model.named_parameters():
                grads = parameter.grad.abs().view(-1).cpu().numpy()
                for i, grad in enumerate(grads):
                    sensitivities[(name, i)] = grad
            return sensitivities

    def get_maskIds(self, sensitivity_values_node, weight_id_map, sensitive_percentage):
        num_weights = len(sensitivity_values_node)
        top_k = int(np.ceil(sensitive_percentage * num_weights / 100))
        self.Mask_Number = top_k
        sorted_weights = sorted(sensitivity_values_node.items(), key=lambda item: item[1], reverse=True)[:top_k]
        weights = [weight for (layer, index), weight in sensitivity_values_node.items()]
        top_weight_ids = [weight_id_map[layer][index] for (layer, index), _ in sorted_weights]
        return top_weight_ids, weights

    def calculate_common_ids(self, index1, index2):
        arr1 = self.maskIds[index1]
        arr2 = self.maskIds[index2]
        sarr1 = set(arr1)
        sarr2 = set(arr2)
        inter = sarr1.intersection(sarr2)
        similarity1 = len(inter)
        return similarity1

    def calculate_distance(self):
        similarity_matrix = np.zeros((self.num_nodes, self.num_nodes))

        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                if DISTANCE_METRIC == "coordinate":
                    similarity = self.calculate_common_ids(i, j)
                elif DISTANCE_METRIC == "cosine":
                    if JUST_COMPARE_SIGNIFICANCE:
                        np_grad_i = np.array(self.grads[i])
                        np_grad_j = np.array(self.grads[j])
                        grad_i_significant_indices = self.get_significant_weights_indices(np_grad_i)
                        grad_j_significant_indices = self.get_significant_weights_indices(np_grad_j)
                        grad_i_significant_weights = np_grad_i[grad_i_significant_indices]
                        grad_j_significant_weights = np_grad_j[grad_j_significant_indices]
                        similarity = 1 - cosine(grad_i_significant_weights, grad_j_significant_weights)
                    else:
                        similarity = 1 - cosine(self.grads[i], self.grads[j])
                else:
                    raise ValueError(f"Unsupported distance metric: {DISTANCE_METRIC}")
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity
            similarity_matrix[i, i] = self.Mask_Number
        distances = self.Mask_Number - similarity_matrix

        self.save_distances_to_csv(distances)
        print(distances)
        self.plot_distance_matrix(distances)

        return distances

    def plot_distance_matrix(self, distances):
        plt.figure(figsize=(10, 8))
        sns.heatmap(distances, annot=True, fmt=".4f", cmap="YlOrRd",
                    xticklabels=range(self.num_nodes), yticklabels=range(self.num_nodes),
                    vmin=0, vmax=np.max(distances))
        plt.title("Distance Matrix")
        plt.xlabel("Client Index")
        plt.ylabel("Client Index")
        plt.show()

    def make_clusters(self):
        normal_distances = (self.distances + self.distances.T) / 2
        np.fill_diagonal(normal_distances, 0)
        affinity_propagation = AffinityPropagation(affinity="precomputed", random_state=0)
        normal_distances = -normal_distances
        clusters = affinity_propagation.fit_predict(normal_distances)
        print(f"cluster results:{clusters}")
        max_label = max(clusters)
        noise_indices = clusters == -1
        unique_noise_labels = np.arange(max_label + 1, max_label + 1 + np.sum(noise_indices))
        clusters[noise_indices] = unique_noise_labels
        cluster_list = [np.where(clusters == cluster_id)[0].tolist() for cluster_id in range(find_num_cluster(clusters))]
        cluster_list = self.index_to_value(cluster_list)
        return cluster_list

    def save_distances_to_csv(self, distances):
        filename = f"distances_{DISTANCE_METRIC}.csv"
        with open(filename, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Node"] + [f"Node_{i}" for i in range(self.num_nodes)])
            for i, row in enumerate(distances):
                writer.writerow([f"Node_{i}"] + row.tolist())
        print(f"Distance matrix saved to {filename}")

    def get_significant_weights_indices(self, arr):
        num_elements = len(arr)
        num_significant = int(np.ceil(num_elements * self.percentage / 100))
        if num_significant == 0:
            return np.array([], dtype=int)
        significant_indices = np.argpartition(-arr, num_significant - 1)[:num_significant]
        significant_indices = significant_indices[np.argsort(-arr[significant_indices])]
        return significant_indices

In [16]:


def model_parameters_to_numpy(path):
    # Load the model's state dictionary
    model = torch.load(path, map_location=torch.device('cpu'))

    # Initialize an empty list to store parameters
    params = []

    # Iterate through the model to extract parameters
    for name, param in model.items():
        if 'bias' not in name:  # Exclude bias terms (optional)
            params.append(param.detach().cpu().numpy())  # Convert to numpy array

    # Return the parameters as a numpy array
    return np.array(params, dtype=object)


In [17]:
def top_to_one_other_zero(arr, P):
    flat_arr = arr.flatten()
    threshold_value = np.percentile(flat_arr, P * 100)

    # Create a new array where values below the threshold are 0 and others are 1
    result = np.where(arr <= threshold_value, 0, 1)

    return result

In [18]:
def group_clients_by_cluster(labels):
    clusters = {}
    for client_id, cluster_id in enumerate(labels):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(client_id)
    return clusters

In [19]:
import numpy as np
from collections import defaultdict
def cluster_to_list(arr):
    cluster_dict = defaultdict(list)
    for index, cluster in enumerate(arr):
        cluster_dict[cluster].append(index)
    clusters = [indices for _, indices in sorted(cluster_dict.items())]

    return clusters

## Cosine

In [20]:
# N = len(params)
# print("Number of parameters (N):", N)

# Cosine_similarity = np.zeros((N, N))

# for i in range(N):
#     for j in range(i + 1, N):
#         # Reshape the 1D arrays to 2D arrays
#         param_i = params[i].reshape(1, -1)
#         param_j = params[j].reshape(1, -1)
#         # Extract the scalar value from the 2D array returned by cosine_similarity
#         Cosine_similarity[i][j] = cosine_similarity(param_i, param_j).item()

# # Make the matrix symmetric (copy upper triangle to lower triangle)
# Cosine_similarity = Cosine_similarity + Cosine_similarity.T

# # Set diagonal elements to 1 (a vector is perfectly similar to itself)
# np.fill_diagonal(Cosine_similarity, 1)

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(Cosine_similarity, annot=True, fmt=".4f", cmap="YlGnBu",
#             xticklabels=range(N), yticklabels=range(N), vmin=-1, vmax=1)
# plt.title("Cosine Similarity Matrix")
# plt.xlabel("Parameter Index")
# plt.ylabel("Parameter Index")
# plt.show()

In [21]:
# ap = AffinityPropagation(affinity='precomputed', random_state=0).fit(Cosine_similarity)
# cosine_labels = ap.labels_

# clusters = group_clients_by_cluster(cosine_labels)

# print("Clients clustering based on their dataset:", clusters)
# print("Cluster labels for each parameter:", cosine_labels)

**Coordinate-Based**

In [ ]:
# coordinates = [top_to_one_other_zero(params[i], P) for i in range(N)]
# for i in range(N):
#   tops=top_to_one_other_zero(params[i], P)
#   coordinates.append(tops)
# ID_similarity=np.zeros((N,N))
# for i in range(N):
#   for j in range(i+1,N):
#     ID_similarity[i][j]=cosine_similarity(coordinates[i].reshape(1, -1),coordinates[j].reshape(1, -1))

# ID_distance = -ID_similarity


# ID_similarity = np.zeros((N, N))

# # Compute cosine similarity for each pair of coordinates
# for i in range(N):
#     for j in range(i + 1, N):
#         similarity = cosine_similarity(
#             coordinates[i].reshape(1, -1),
#             coordinates[j].reshape(1, -1)
#         )
#         ID_similarity[i][j] = similarity.item()  # Use .item() to extract the scalar value

# Make the matrix symmetric (copy upper triangle to lower triangle)
# ID_similarity = ID_similarity + ID_similarity.T

clusters = Clustering([0,1,2,3,4,5,6,7,8,9], train_loaders, SENSITIVITY_PERCENTAGE).Clusters

<ipython-input-15-e1adfd1caae3>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_path)


Model weights and sensitivity data for client #0 processed.
Model weights and sensitivity data for client #1 processed.
Model weights and sensitivity data for client #2 processed.
Model weights and sensitivity data for client #3 processed.
Model weights and sensitivity data for client #4 processed.
Model weights and sensitivity data for client #5 processed.


In [ ]:
# ap = AffinityPropagation(affinity="precomputed", random_state=0).fit(ID_distance)
# ID_labels = ap.labels_
# # ID_similarity_show = np.round(ID_similarity, 2)
# print(ID_labels)

In [ ]:
# Cosine_similarity_show=np.round(Cosine_similarity, 2)
# print(cosine_labels)
# Cosine_similarity

In [ ]:
# print(ID_labels)
# ID_similarity

In [ ]:
# matrix1=Cosine_similarity.flatten()
# matrix2=ID_similarity.flatten()
# from scipy.stats import pearsonr, spearmanr

# corr_pearson, _ = pearsonr(matrix1, matrix2)
# corr_spearman, _ = spearmanr(matrix1, matrix2)
# mae = np.mean(np.abs(matrix1 - matrix2))
# diff=matrix1- matrix2
# min_,max_,avg_=np.min(diff),np.max(diff),np.average(diff)
# corr_pearson, corr_spearman, mae, min_,max_,avg_

In [ ]:
# matrix2

In [ ]:
# import numpy as np
# from sklearn.linear_model import LinearRegression

# def find_regression_coefficients(x, y):
#     x = np.array(x).reshape(-1, 1)
#     y = np.array(y)

#     # Create and fit the regression model
#     model = LinearRegression()
#     model.fit(x, y)

#     return model.coef_[0], model.intercept_
# slope, intercept = find_regression_coefficients(matrix1, matrix2)
# cofficient=f"cosine(x,y)={round(slope,3)} ID_sim(x,y) {round(intercept,3)}"
# cofficient

In [ ]:
# # Filter non-zero values for both matrices
# non_zero_indices_x = np.nonzero(matrix1)
# non_zero_indices_y = np.nonzero(matrix2)

# # Get non-zero values
# x_non_zero = matrix1[non_zero_indices_x]
# y_non_zero = matrix2[non_zero_indices_y]

# # Create scatter plot for non-zero values
# plt.figure(figsize=(8, 6))

# # Plot X values (Cosine Similarity) in blue
# plt.scatter(range(len(x_non_zero)), x_non_zero, color='blue', alpha=0.7, label='Cosine Similarity (X)')

# # Plot Y values (Approximation Similarity) in red
# plt.scatter(range(len(y_non_zero)), y_non_zero, color='red', alpha=0.7, label='Approximation Similarity (Y)')

# # Add labels and title
# plt.xlabel('Data Index', fontsize=12)
# plt.ylabel('Similarity Values', fontsize=12)
# plt.title(f"Pearson Correlation: {round(corr_pearson,3)} Spearman Correlation: {round(corr_spearman,3)} "+cofficient, fontsize=14)
# plt.legend()
# plt.grid(alpha=0.4)

# plt.show()





## **Clustering and Silouhette**

In [ ]:
# import numpy as np
# from sklearn.metrics import silhouette_score
# from sklearn.cluster import AffinityPropagation
# from sklearn.metrics import pairwise_distances
# data = np.array([
#     [6000, 0, 3000, 0, 0, 0, 0, 0, 0, 0],
#     [0, 1500, 0, 0, 0, 0, 0, 0, 0, 3000],
#     [0, 1500, 3000, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 3000, 3000, 0, 0, 0, 0, 0],
#     [0, 1500, 0, 0, 3000, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 0, 6000, 0, 2000, 0, 0],
#     [0, 0, 0, 0, 0, 0, 3000, 2000, 0, 0],
#     [0, 0, 0, 0, 0, 0, 3000, 2000, 0, 0],     # Added rows assuming the pattern.
#     [0, 0, 0, 3000, 0, 0, 0, 0, 6000, 0],
#     [0, 1500, 0, 0, 0, 0, 0, 0, 0, 3000]
# ])

# # Calculate cosine similarity
# cosine_dist_opt = pairwise_distances(data, metric='cosine')
# cosine_similarities_opt = -cosine_dist_opt  # Convert distances to negative similarities for AP

# # Determine the preference based on the cosine similarity matrix
# preference = np.median(cosine_similarities_opt)

# # Affinity Propagation with cosine similarity
# clustering = AffinityPropagation(affinity='precomputed', preference=preference, random_state=42).fit(cosine_similarities_opt)

# cluster_labels = clustering.labels_
# # Calculate the silhouette score


# silhouette_avg_opt = silhouette_score(data[:9], cluster_labels, metric='cosine')
# silhouette_avg_cosine = silhouette_score(data, cosine_labels, metric='cosine')
# silhouette_avg_method = silhouette_score(data, ID_labels, metric='cosine')


# # Organizing the final clusters as [[], [], ...]
# num_clusters = len(np.unique(cluster_labels))
# clusters = [[] for _ in range(num_clusters)]
# for index, label in enumerate(cluster_labels):
#     clusters[label].append(index)

# # Output the clusters and silhouette score
# print(f"Silhouette Score with[{clusters}]: {silhouette_avg_opt}")
# print(f"Silhouette Score_cosine with [{cluster_to_list(cosine_labels)}]: {silhouette_avg_cosine}")
# print(f"Silhouette Score_ID method with [{cluster_to_list(ID_labels)}]: {silhouette_avg_method}")


## Others

In [ ]:
# similarity_vector=[]
# for p in range (0,100):
#     similarity_vector.append(cosine_similarity(params[1], array_prune(params[1], p/100)))

In [ ]:
# similarity_vector_ID=[]
# for p in range (1,100):
#     similarity_vector_ID.append(cosine_similarity(params[1], top_to_one_other_zero(params[1], p/100)))

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# values = np.random.rand(100)
# plt.figure(figsize=(10, 5))  # Optional: Adjust the figure size
# plt.plot(similarity_vector, marker='o', linestyle='-', label='similarity of prun')
# plt.plot(similarity_vector_ID, marker='o', linestyle='-', label='similarity of ID')
# plt.title('Plot of 100 Values')
# plt.xlabel('Index')
# plt.ylabel('Value')
# plt.legend()
# plt.grid(True)
# plt.show()